In [1]:
from csoundengine import *

In [2]:
s = Session()

--Csound version 7.0 (double samples) May 14 2025
[commit: ba226ccf9e9d38cd4516498e6ff7d59ba734b5e1]
libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Analog Stereo: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Analog Stereo:
SECTION 1:


In [3]:
s.defInstr('busoscil', r'''
|ibus|
kfreq = busin:k(ibus)
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(busoscil, ibus=0)

In [12]:
bus = s.assignBus(value=1000, persist=True)
synth = s.sched('busoscil', ibus=bus.token)
automation = bus.automate((0, 800, 3, 500), overtake=False)
synth

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 busoscil=401.0006 start=35.955 dur=-1.000 p5=4)

In [13]:
s.engine.unsched(automation)

In [14]:
synth.stop()

In [4]:
import sndfileio
import time
t0 = time.time()
info = sndfileio.sndinfo('../snd/piano-pitchtest-2.flac')
t1 = time.time()
print(info, t1 - t0)

samplerate : 44100
nframes    : 459938
channels   : 1
encoding   : pcm16
fileformat : flac
duration   : 10.429 0.0013918876647949219


In [3]:
tab = s.readSoundfile('../snd/piano-pitchtest-2.flac')
print(t1 - t0)
s.defInstr('test0', r'''
kfreq = p5
outch 1, oscili:a(0.1, lag(kfreq, 0.01))
''')
s.sched('test0', 0, 2, kfreq=440)
ev = s.playSample(tab)
ev

0.7673156261444092


Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 .playSample=402.0002 start=4.059 dur=10.449 p5=301 isndtab=0 istart=0.02 ifadein=0.02 ifadeout=1 kchan=1 kspeed=1 kgain=0.5 kpan=-1)

In [13]:
s.defInstr('test1', r'''
|kfreq|
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test1, kfreq=0)

In [14]:
name = 'test5'
s.defInstr(name, r'''
ibus = p5
kfreq = busin:k(ibus)
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test5, ibus:5)

In [15]:
bus = s.assignBus(value=1000)
print(bus)
synth = s.sched(name, ibus=bus.token)
bus.automate((0, 800, 3, 500), overtake=False)
synth

Bus('control', token=4)


Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 test5=504.0002 start=31.429 dur=-1.000 p5=4)

In [16]:
synth.stop()

In [20]:
e = s.engine

buses = [e.assignBus(value=600) for _ in range(200)]
e.sync()
for bus in buses:
    synth = s.sched('test5', dur=1, ibus=bus)
    e.writeBus(bus, 700, 0.001)
    

In [23]:
s.defInstr('test', r'''
  ibus = p5
  kfreq = busin:k(ibus)
  outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')

buses = [s.assignBus(value=2000) for _ in range(200)]
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus.token)
    bus.set(1000, 0)
    

In [24]:
e.writeBus(bus.token, 1500)

In [28]:
bus.set(1600)

In [18]:
e.unschedAll()

end of section 1	 sect peak amps:  0.87595  0.00000
SECTION 1:


In [20]:
s.defInstr('sender', r'''
ibus = p5
ifreqbus = p6
kfreq = busin:k(ifreqbus)
asig oscili 0.1, lag:k(kfreq, 0.01)
busout(ibus, asig)
''')

s.defInstr('receiver', r'''
ibus  = p5
kgain = p6
asig = busin:a(ibus)
asig *= a(kgain)
outch 1, asig
''')


Instr(receiver, ibus:5, kgain:6)

In [27]:
bus = s.assignBus()
freqbus = s.assignBus(value=600)

chain = SynthGroup([s.sched('sender', ibus=bus, ifreqbus=freqbus),
                    s.sched('receiver', ibus=bus, kgain=0.5)])
# automate wins over set
freqbus.set(1500, delay=1.5)
freqbus.automate((0, 0, 1, 400), delay=1, overtake=True)

chain

Button(description='Stop', style=ButtonStyle())

Output()

p1,start,dur,p4,ibus,ifreqbus
501.0018 𝍪,0.034,-1.000,0,32,33
p1,start,dur,p4,ibus,kgain
502.0018 𝍪,0.034,-1.000,0,32,0.5


In [15]:
s.unschedAll()